In [1]:
import secrets
import math

Return GCD of 2 numbers

In [3]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

## Continued Fractions

#### <b>def rational_to_contfrac(p,q)</b>
Converts rational fraction p/q into list of partial quotients [a0, .... , an] <br>

#### <b>def rational_to_contfrac(p,q)</b>
Compute list of convergents using list of partial quotients <br>

#### <b> def contfrac_to_rational(frac) </b>
Converts finite continued fraction [a0, .... , an] to a p/q rational fraction




In [4]:
def rational_to_contfrac(p ,q):
    a = p//q
    quotients = [a]
    while a * q != p:
        p,q = q, p-a*q
        a = p//q
        quotients.append(a)
    return quotients


def convergents_from_contfrac(cf):
    convergents = []
    for i in range(len(cf)):
        convergents.append(contfrac_to_rational(cf[0:i]))
    
    return convergents


def contfrac_to_rational(cf):
    if len(cf) == 0:
        return (0,1)
    
    num = cf[-1]
    denominator = 1 

    for i in range(-2, -len(cf)-1, -1):
        num, denominator = cf[i]*num+denominator, num
    
    return (num, denominator)

Testing the functions

In [18]:
contfrac = rational_to_contfrac(34, 99)
print(contfrac)

conv = convergents_from_contfrac(contfrac)
print(conv)

rational = contfrac_to_rational(contfrac)
print(rational)

[0, 2, 1, 10, 3]
[(0, 1), (0, 1), (1, 2), (1, 3), (11, 32)]
(34, 99)


Rabin-Miller Primality Test <br>
Check if number is prime, p is the number to be tested and s is the number of rounds <br>
S should ensure probabilitiy of p not being prime is < 2^(-80)

In [19]:
def RMPT(p, s):
    q = p - 1
    k = 0 
    
    # if newP % 2 have remainder then stop
    while q % 2 != 1: 
        q //= 2 
        k += 1 

    for i in range(s):
        #Generate random a 
        a = secrets.randbelow(p-2) + 2 

        while gcd(a,p) != 1:
            a = secrets.randbelow(p-2) + 2
            
        z = pow(a, q, p) # z = a^q mod p

        # checking a^q != 1 mod p and a^q != -1 mod p 
        if z != 1 and z != p-1:
            for j in range(1, k):
                z = pow(z, 2, p)
                if z == 1:
                    return False

            if z != p-1:
                return False
    return True

In [20]:
# Check the result 
print(RMPT(58536828032471131348456426324506659436967195846753702250333704608966791528534067798278728747981548972696886762937373961684727824495513261968904996453134494716095278694362125703914706608360435446792042241177272588181082115012537677828367957226772929218486970585153142173891310402614698182537457744036891034121, 2))

False


Generate primes p and q 

In [21]:
def generate_prime(bits, rounds):
    while(True):
        num = secrets.randbits(bits) # using CSRPNG to generate

        # if number is even make it odd 
        if num % 2 == 0:
            num += 1

        elif num % 3 == 0:
            continue 

        elif num % 5 == 0:
            continue

        elif num % 7 == 0:
            continue

        elif num % 11 == 0: 
            continue

        elif RMPT(num, rounds):
            return num


RSA generate weak decryption key using public key e and 2 primes p and q

In [22]:
def RSA_generate_weakd(p, q):
    N = p*q
    d = secrets.randbelow(math.isqrt(math.isqrt(N)) // 3) + 2
    
    phiN = (p-1) * (q-1)
    gcd_val = gcd(d, phiN)

    while gcd_val != 1:
        d = secrets.randbelow(math.isqrt(math.isqrt(N)) // 3) + 2
        gcd_val = gcd(d, phiN)
    
    print("Weak d = ", d)
    print("Weak d bit length: ", d.bit_length())

    return d


In [23]:
def inv(a, m):
    m0 = m
    x0 = 0
    x1 = 1
    if m == 1:
        return 0
    # Apply extended Euclid Algorithm
    while a > 1:
        q = a // m  # q is quotient

        t = m
        # m is remainder now, process same as euclid's algo
        m = a % m
        a = t

        t = x0

        x0 = x1 - q * x0

        x1 = t

    # Make x1 positive
    if x1 < 0:
        x1 = x1 + m0

    return x1


Encrypt with public key e

In [24]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Decrypt using private key d

In [25]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

In [26]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [27]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

## RSA 1024
User of RSA encrypt message using RSA 1024

Generate primes p and q 

User deliberately generate a weak d exponent by setting max bit length to 1/3 * N^1/4 to get faster decryption 

Encryption exponent e (1024 bit) is obtained by inversing weak d 


In [29]:
phi_test = False

while phi_test == False: 
    p = generate_prime(512, 3)
    q = generate_prime(512, 3)
    
    phiN = (p-1) * (q-1)
    N = p * q

    d = RSA_generate_weakd(p, q)
    e = inv(d, phiN)

    if gcd(e, phiN) == 1:
        phi_test = True
    

    print('E is: ', e)
    print('E bit length: ', e.bit_length())

Weak d =  1508588693260847212567844171720829540523113200195872643370165515304637646203
Weak d bit length:  250
E is:  30581472026331650198030073149183420303670165898854645900649085643966297355275803748450279367810277015442512015771005873341576652681360064052273708001720275962845056738029482745982859994751356644035705211483356611984082776005984784280562510742978507545253389626405158025532984425615264285878742922113509300191
E bit length:  1022


In [30]:
message = "Your Auntie"

ascii_message = convert_to_ascii(message)
ascii_message.bit_length()

encrypted_message = RSA_encrypt(ascii_message, e, N)
print('Encrypted message in ascii: ', encrypted_message)
print('Encrypted message in String: ', convert_to_string(encrypted_message))



Encrypted message in ascii:  134178334573442195028495388061517740015630743205964170088616651442900492748739957973542882863312586462757462837314834866981060018375704352585299405203736567703766522814080056072617714830121697882214346068996635305311787783530535308476951918580606086549945536884252971401314748304296167740025489980760343293465
Encrypted message in String:  ²ŎȽƺÃǯƄ=ȅˤɶ˧ÍτªXɨʋƺ΄ǬˬˣνύȞͲ͟ĸɊǎ˵ǎͅĺ͂͢ϕ<ŷˀŠɉīƕËˠȷʿ˾Ȋ̮P8Hɩˊ̾yʹͲÖŚDϤɻıķ̓̏ȒȗĴǜηΖɄɞVȥαȘʹüϋƑĺˬİĨ§ˤǩϔ˸ŗĥǑ


In [31]:
decrypted_message = RSA_decrypt(encrypted_message, d, N)

print('Decrypted msg in ascii: ', decrypted_message)
print('Decrypted msg in string: ', convert_to_string(decrypted_message))

Decrypted msg in ascii:  89111117114032065117110116105101
Decrypted msg in string:  Your Auntie


In [32]:
def bitlength(x):
    '''
    Calculates the bitlength of x
    '''
    assert x >= 0
    n = 0
    while x > 0:
        n = n+1
        x = x>>1
    return n

In [33]:
def isqrt(n):
    '''
    Calculates the integer square root
    for arbitrary large nonnegative integers
    '''
    if n < 0:
        raise ValueError('square root not defined for negative numbers')
    
    if n == 0:
        return 0
    a, b = divmod(bitlength(n), 2)
    x = 2**(a+b)
    while True:
        y = (x + n//x)//2
        if y >= x:
            return x
        x = y

In [34]:
def is_perfect_square(n):
    '''
    If n is a perfect square it returns sqrt(n),
    
    otherwise returns -1
    '''
    h = n & 0xF; #last hexadecimal "digit"
    
    if h > 9:
        return -1 # return immediately in 6 cases out of 16.

    # Take advantage of Boolean short-circuit evaluation
    if ( h != 2 and h != 3 and h != 5 and h != 6 and h != 7 and h != 8 ):
        # take square root if you must
        t = isqrt(n)
        if t*t == n:
            return t
        else:
            return -1
    
    return -1

## Wiener Attack 

Attacker only knows public parameters {n, e} and does not know any other parameters 

Goal: To check if implementation by user is careless and get the decryption exponent d to compromise encryption

In [35]:
def wiener_attack(e, n):
    frac = rational_to_contfrac(e, n)
    convergents = convergents_from_contfrac(frac)

    for (k ,d) in convergents:

        if k!=0 and (e*d-1) % k == 0:
            phi = (e*d-1) // k
            s = n - phi + 1

            discr = s * s - 4 * n

            if discr >= 0:
                t = is_perfect_square(discr)
                if t!= -1 and (s+t) % 2 == 0:
                    print("Hacked")
                    return d

In [36]:
times = 5

while times > 0:
    hacked_d = wiener_attack(e, N)

    if d == hacked_d:
        print('Hack Worked')
        hack_decrypted_message = RSA_decrypt(encrypted_message, hacked_d, N)
        print(convert_to_string(hack_decrypted_message))
        break
    else:
        print('Hack Failed')

    times -= 1

Hacked
Hack Worked
Your Auntie
